In [96]:
import cupy as cp
import plotly.graph_objects as plt
from prettyprint import *

maxPos = 8
minPos = -8
stepPos = 0.25

def freePropagator(finPos, startPos, finTime, startTime=0, m=1):
    return cp.sqrt(m / (2 * cp.pi * 1j * (finTime - startTime))) * cp.exp(1j * m / (2 * (finTime - startTime)) * (finPos - startPos)**2)

def freePropagatorC(qf, qfp, q0, q0p, tf, t0):
    return freePropagator(qf, q0, tf, t0) + freePropagator(qfp, q0p, tf, t0)

def initStateFunction(q0, q0p, σ1, s1, p1, σ2, s2, p2):
    return (1/2 * cp.pi * σ1)**(1/4) * cp.exp(-(q0 - s1)**2 / (4 * σ1**2) + 1j * p1 * q0) * (1/2 * cp.pi * σ2)**(1/4) * cp.exp(-(q0p - s2)**2 / (4 * σ2**2) + 1j * p2 * q0p)

def posToIndex(pos):
    return int((pos - minPos) / stepPos)

def initState(q):
    return initStateFunction(q[0], q[1], 0.5, 1, 0, 0.5, -1, 0)

def springPropagator1(qf, qfp, q0, q0p, tf, t0):
    return freePropagatorC(qf, qfp, q0, q0p, tf, t0) * (1 - 1j * α * tf / 6 * (-2 * (m * (q0 + qf)**2 + m * (q0p + qfp)**2 + 1j * tf) + 2*q0*q0p + q0*qfp + q0p*qf + 2*qf*qfp))

In [97]:
pos1Vect = cp.arange(minPos, maxPos + stepPos, stepPos)
pos2Vect = cp.arange(minPos, maxPos + stepPos, stepPos)
posVectSize = len(pos1Vect)

posMat = cp.dstack(cp.meshgrid(pos1Vect, pos2Vect)).reshape(-1, 2)
initMat = cp.round(cp.array([initState(q) for q in posMat]), 6).reshape(posVectSize, posVectSize)
initProb = (abs(initMat)**2).tolist()

α = 0.02
m = 1

In [98]:
dat = plt.Contour(z = initProb, x = pos1Vect.tolist(), y = pos2Vect.tolist())
fig = plt.Figure(data = dat)
fig.update_layout(yaxis_scaleanchor= "x")
fig.show()

In [100]:
springPropagator1(1, 1, 0, 0, 1, 0) * initMat[32, 32]

(0.09139315697145269-0.026150610581168805j)

In [ ]:
finalTime = 0.0475
finalMat = Matrix{ComplexF32}(undef, posVectSize, posVectSize)

for xf in pos1Vect, xfp in pos2Vect
    sumPos = 0
    for i in 1:posVectSize, j in 1:posVectSize
        x0 = pos1Vect[i]
        x0p = pos2Vect[j]
        sumPos += springPropagator1(xf, xfp, x0, x0p, finalTime, 0) * initMat[i, j]
    end
    finalMat[posToIndex(xf), posToIndex(xfp)] = sumPos
end
currentPlot = surface(pos1Vect, pos2Vect, round.(normalize(abs2.(finalMat)), digits = 6))

In [95]:
finalTime = 0.0475
finalMat = cp.empty(shape = (posVectSize, posVectSize), dtype = complex)

for xf in pos1Vect:
	for xfp in pos2Vect:
		sumPos = 0
		for i in range(0, posVectSize):
			for j in range(0, posVectSize):
				x0 = pos1Vect[i]
				x0p = pos2Vect[j]
				sumPos += springPropagator1(xf, xfp, x0, x0p, finalTime, 0) * initMat[i, j]
		finalMat[posToIndex(xf), posToIndex(xfp)] = complex(sumPos)

KeyboardInterrupt: 

In [101]:
finalMat

array([[46.91588669+32.82604909j, 51.13449203+28.81307968j,
        54.51578948+25.11983186j, ...,  0.         +0.j        ,
         0.         +0.j        ,  0.         +0.j        ],
       [ 0.         +0.j        ,  0.         +0.j        ,
         0.         +0.j        , ...,  0.         +0.j        ,
         0.         +0.j        ,  0.         +0.j        ],
       [ 0.         +0.j        ,  0.         +0.j        ,
         0.         +0.j        , ...,  0.         +0.j        ,
         0.         +0.j        ,  0.         +0.j        ],
       ...,
       [ 0.         +0.j        ,  0.         +0.j        ,
         0.         +0.j        , ...,  0.         +0.j        ,
         0.         +0.j        ,  0.         +0.j        ],
       [ 0.         +0.j        ,  0.         +0.j        ,
         0.         +0.j        , ...,  0.         +0.j        ,
         0.         +0.j        ,  0.         +0.j        ],
       [ 0.         +0.j        ,  0.         +0.j        